author = 'Ricardo Pasquini'
# Ejemplo Georeferenciación básica basada en direcciones

## 1. Importacion de librerias y levanto direcciones

In [65]:
import pymongo
client = pymongo.MongoClient('localhost', 27017)
db = client["vivienda"]  #vivienda is the name of the db

In [1]:
import pandas as pd
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
print(currentdir)
#write a for-loop to open many files -- leave a comment if you'd #like to learn how
filename = "Demanda_SgoEstero.xlsx" 
df=pd.read_excel(currentdir+'\\datos_vivienda_social\\'+filename)
df=df.rename(index=str, columns={" DOMICILIO  ": "DIRECCION", " NDOC  ": 'DNI_TITULAR'})

C:\Users\Richard\Dropbox\projects\vivienda


## 2. Preparacion de las direcciones 

In [3]:
#Separar el barrio en una columna aparte
def agregarbarrio(df):
    df['Barrio']=""
    for index, row in df.iterrows():
        fulladdress=row['DIRECCION']
        #print('fulladdress', fulladdress)
        try:
            index_barrio=fulladdress.index('Bº')
            barrioname=fulladdress[index_barrio+2:]
            dire0=fulladdress[:index_barrio]
            #print(barrioname)
        except ValueError:
            barrioname=""
            dire0=fulladdress
        df.at[index,'Barrio']=barrioname
        df.at[index,'DIRECCIONsinBarrio']=dire0
        
    return df

#limpiar DPTO que ensucia la ubicacion
def borrarDPTO(df):
    for index, row in df.iterrows():
        diresinbarrio=row['DIRECCIONsinBarrio']
        try:
            index_DPTO=diresinbarrio.index('DPTO')
            dire0=diresinbarrio[:index_DPTO]
                #print(barrioname)
        except ValueError:
            dire0=diresinbarrio
        df.at[index,'DIRECCIONsinBarrio']=dire0
    return df




In [4]:
df=agregarbarrio(df)
df=borrarDPTO(df)
#df=arreglarcalle(df)

In [6]:
df['DIRECCIONsinBarrio']=df['DIRECCIONsinBarrio'].str.replace('C/', 'CALLE ')
df['DIRECCIONsinBarrio']=df['DIRECCIONsinBarrio'].str.replace('MZA. ', 'MANZANA ')
df['DIRECCIONsinBarrio']=df['DIRECCIONsinBarrio'].str.replace('MZA.', 'MANZANA ')
df['DIRECCIONsinBarrio']=df['DIRECCIONsinBarrio'].str.replace('MZA ', 'MANZANA ')

In [7]:
print(df['DIRECCIONsinBarrio'].to_string())

0                                 CALLE  16 Y 103 Nº 407 
1                                        CALLE 69 Nº 829 
2                               CALLE  DEMOCRACIA Nº 326 
3                                      RIVADAVIA Nº 1605 
4                                     MANZANA 10.LOTE.11.
5              CALLE LAMADRID Y PJE/Vº DE LORETO Nº 2332 
6                                2º PJE Y CALLE 489 S/Nº 
7                                        CALLE 65 Nº 362 
8                                 CALLE AZCUENAGA Nº 164 
9                                       CERVANTES Nº 237 
10                            CALLE PERALTA LUNA Nº 1374 
11                               24 DE SEPTIEMBRE Nº 892 
12                                    MANZANA 44 LOTE 22 
13                               AV BELGRANO Nº 1379 1ºP/
14                                CALLE  REDUCCION Nº 55 
15                                SAN MARTIN Nº 185 3º C 
16                                     MANZANA 3 LOTE 17 
17            

In [123]:
df['Barrio']=df['Barrio'].str.replace('LA BDA', ', LA BANDA,')
df['Barrio']=df['Barrio'].str.replace('EJTO ARG', 'EJERCITO ARGENTINO ')

In [124]:
print(df['Barrio'].to_string())

0                                  BORGES  154971383
1                                EJERCITO ARGENTINO 
2                                        LA CATOLICA
3                                          RIVADAVIA
4                                          SIGLO XXI
5                                              STA L
6                                 SAN JOSE DE FLORES
7                                EJERCITO ARGENTINO 
8                               BELGRANO/CONDICIONAL
9                              SAN MARTIN LA BANDA, 
10                                          M.MORENO
11                                          BELGRANO
12                                C. CONTRERAS OESTE
13                                        R CARRILLO
14                                          LIBERTAD
15                                            CENTRO
16                          VILLA DEL CARMEN 1º AMP.
17                                 C CONTRERAS VIEJO
18                                          EJ

## Guardando la data original arreglada

In [11]:
import pickle
with open(currentdir+'\\dataframes\\demanda_santiago_original.pickle', 'wb') as f:
        pickle.dump(df,f)


In [97]:
import geocoder

In [133]:
#direprueba="VIAMONTE Nº 15, BARRIO RAMON CASTILLO, SANTIAGO DEL ESTERO"
#direprueba="CERVANTES Nº 237 , LA BANDA"
direprueba="BALCARCE 1039, Barrio LA CATOLICA, Santiago del Estero"

In [129]:
coordenadas_santiago=[-27.783613, -64.264826]

In [137]:
g=geocoder.arcgis(direprueba, proximity=coordenadas_santiago)
g.json

{'address': 'Balcarce 1039, Santiago del Estero, G4200, Santiago del Estero',
 'bbox': {'northeast': [-27.79671448709732, -64.24118274748962],
  'southwest': [-27.79871448709732, -64.24318274748963]},
 'confidence': 9,
 'lat': -27.79771448709732,
 'lng': -64.24218274748962,
 'ok': True,
 'quality': 'StreetAddressExt',
 'raw': {'name': 'Balcarce 1039, Santiago del Estero, G4200, Santiago del Estero',
  'extent': {'xmin': -64.24318274748963,
   'ymin': -27.79871448709732,
   'xmax': -64.24118274748962,
   'ymax': -27.79671448709732},
  'feature': {'geometry': {'x': -64.24218274748962, 'y': -27.79771448709732},
   'attributes': {'Score': 96.78, 'Addr_Type': 'StreetAddressExt'}}},
 'score': 96.78,
 'status': 'OK'}

In [ ]:
#pequeña funcion para devolver el json. 
def geocodeaddress(addressstring):  #I am currently using Arcgis service via geocode library
    g = geocoder.arcgis(addressstring)
    print(g.json)

    return g.json

In [ ]:
geocoderesults=geocodeaddress(addressstring)

In [144]:
from itertools import islice
LIMIT = 4
# iterrows and unpacking
#for index, row in islice(df.iterrows(), LIMIT):
for index, row in df.iterrows():
        print(index)
        addressstring=row['DIRECCIONsinBarrio']+", BARRIO "+row['Barrio']+', Santiago del Estero'
        #print(addressstring)
        geocoderesults=geocoder.arcgis(addressstring, proximity=coordenadas_santiago)
        #print({'id':index,'DNI_TITULAR':row['DNI_TITULAR'],'geocoderesults':geocoderesults.json})
        # Guardo el resultado en Mongo
        db.santiago.insert({'id':index,'DNI_TITULAR':row['DNI_TITULAR'],'geocoderesults':geocoderesults.json})

0


C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # This is added back by InteractiveShellApp.init_path()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
